# Comprehensive Comparative Analysis: Prompt Engineering for Financial Risk Assessment

**Research Question**: How do different prompt engineering strategies and LLM models perform on financial risk assessment tasks?

## Executive Summary

This notebook provides deep statistical analysis across all 12 risk assessment experiments:
- **4 Prompting Strategies**: Zero-Shot, Few-Shot, Chain-of-Thought (CoT), Tree-of-Thought (ToT)
- **3 Models**: Gemini 2.0 Flash, Gemini 1.5 Flash, Llama-3.3-70B
- **Dataset**: FinancialPhraseBank Sentences_AllAgree.txt (2,217 samples)

## Analysis Framework

1. **Strategy Effectiveness**: Does prompting complexity improve risk assessment accuracy?
2. **Model Comparison**: Which model performs best across different strategies?
3. **Cost-Benefit Analysis**: Trade-offs between performance and complexity
4. **Statistical Significance**: Are performance differences statistically meaningful?
5. **Production Recommendations**: Best approach for deployment

In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn scipy statsmodels plotly -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
from scipy import stats
from scipy.stats import ttest_ind, f_oneway, friedmanchisquare
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (16, 8)

print("✓ Libraries loaded successfully")

## 1. Load All Experimental Results

In [ ]:
# Define experiment metadata
experiments = {
    "R1": {
        "model": "Gemini 2.0 Flash",
        "strategy": "Zero-Shot",
        "pattern": "../Zero_Shot/r1_GPT_OSS_20B_zero_shot_*.csv",
    },
    "R2": {
        "model": "Gemini 1.5 Flash",
        "strategy": "Zero-Shot",
        "pattern": "../Zero_Shot/r2_GPT_OSS_120B_zero_shot_*.csv",
    },
    "R3": {
        "model": "Llama-3.3-70B",
        "strategy": "Zero-Shot",
        "pattern": "../Zero_Shot/r3_Llama_3.3_70B_zero_shot_*.csv",
    },
    "R4": {
        "model": "Gemini 2.0 Flash",
        "strategy": "Few-Shot",
        "pattern": "../Few_Shot/r4_GPT_OSS_20B_few_shot_*.csv",
    },
    "R5": {
        "model": "Gemini 1.5 Flash",
        "strategy": "Few-Shot",
        "pattern": "../Few_Shot/r5_GPT_OSS_120B_few_shot_*.csv",
    },
    "R6": {
        "model": "Llama-3.3-70B",
        "strategy": "Few-Shot",
        "pattern": "../Few_Shot/r6_Llama_3.3_70B_few_shot_*.csv",
    },
    "R7": {
        "model": "Gemini 2.0 Flash",
        "strategy": "Chain-of-Thought",
        "pattern": "../Chain_of_Thought/r7_GPT_OSS_20B_cot_*.csv",
    },
    "R8": {
        "model": "Gemini 1.5 Flash",
        "strategy": "Chain-of-Thought",
        "pattern": "../Chain_of_Thought/r8_GPT_OSS_120B_cot_*.csv",
    },
    "R9": {
        "model": "Llama-3.3-70B",
        "strategy": "Chain-of-Thought",
        "pattern": "../Chain_of_Thought/r9_Llama-3.3-70B_cot_*.csv",
    },
    "R10": {
        "model": "Gemini 2.0 Flash",
        "strategy": "Tree-of-Thought",
        "pattern": "../Tree_of_Thought/r10_GPT_OSS_20B_tot_*.csv",
    },
    "R11": {
        "model": "Gemini 1.5 Flash",
        "strategy": "Tree-of-Thought",
        "pattern": "../Tree_of_Thought/r11_GPT_OSS_120B_flash_tot_*.csv",
    },
    "R12": {
        "model": "Llama-3.3-70B",
        "strategy": "Tree-of-Thought",
        "pattern": "../Tree_of_Thought/r12_Llama_3.3_70B_tot_*.csv",
    },
}

# Load results
print("=" * 100)
print("📁 LOADING ALL EXPERIMENT DATA")
print("=" * 100)

all_data = {}
for exp_id, meta in experiments.items():
    files = sorted(glob(meta["pattern"]))
    if files:
        latest_file = files[-1]
        df = pd.read_csv(latest_file)
        df["experiment"] = exp_id
        df["model"] = meta["model"]
        df["strategy"] = meta["strategy"]
        all_data[exp_id] = df
        print(f"✓ {exp_id}: {os.path.basename(latest_file)} ({len(df)} samples)")
    else:
        print(f"⚠️  {exp_id}: No files found")

print(f"\n✓ Loaded {len(all_data)} experiments")

## 2. Calculate Performance Metrics

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    matthews_corrcoef,
)


def calculate_metrics(df):
    """Calculate comprehensive metrics"""
    valid_df = df[
        df["predicted_sentiment"].isin(["positive", "negative", "neutral"])
    ].copy()

    if valid_df.empty:
        return None

    y_true = valid_df["true_sentiment"]
    y_pred = valid_df["predicted_sentiment"]

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "macro_f1": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted", zero_division=0),
        "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "mcc": matthews_corrcoef(y_true, y_pred),
        "total_samples": len(df),
        "valid_predictions": len(valid_df),
        "error_rate": (len(df) - len(valid_df)) / len(df) if len(df) > 0 else 0,
    }


# Calculate metrics for all experiments
metrics_list = []
for exp_id, df in all_data.items():
    metrics = calculate_metrics(df)
    if metrics:
        metrics.update(
            {
                "experiment": exp_id,
                "model": experiments[exp_id]["model"],
                "strategy": experiments[exp_id]["strategy"],
            }
        )
        metrics_list.append(metrics)

metrics_df = pd.DataFrame(metrics_list)

print("\n" + "=" * 100)
print("📊 PERFORMANCE METRICS SUMMARY")
print("=" * 100)
display(
    metrics_df[
        ["experiment", "model", "strategy", "accuracy", "macro_f1", "mcc", "error_rate"]
    ].round(4)
)

## 3. Research Question 1: Strategy Effectiveness

**Does increasing prompt complexity improve risk assessment performance?**

In [ ]:
print("\n" + "=" * 100)
print("🔬 STRATEGY EFFECTIVENESS ANALYSIS")
print("=" * 100)

# Group by strategy
strategy_analysis = (
    metrics_df.groupby("strategy")
    .agg(
        {
            "accuracy": ["mean", "std", "min", "max"],
            "macro_f1": ["mean", "std", "min", "max"],
            "mcc": ["mean", "std", "min", "max"],
            "error_rate": ["mean", "std", "min", "max"],
        }
    )
    .round(4)
)

print("\nPerformance by Strategy:")
display(strategy_analysis)

# Statistical test - ANOVA
strategies = ["Zero-Shot", "Few-Shot", "Chain-of-Thought", "Tree-of-Thought"]
strategy_groups = {
    s: metrics_df[metrics_df["strategy"] == s]["accuracy"].values for s in strategies
}

f_stat, p_value = f_oneway(*[strategy_groups[s] for s in strategies])
print(f"\n📈 ANOVA Test for Accuracy:")
print(f"  F-statistic: {f_stat:.4f}")
print(f"  p-value: {p_value:.4f}")
print(
    f"  Result: {'Statistically significant' if p_value < 0.05 else 'Not statistically significant'} (α=0.05)"
)

# Post-hoc Tukey HSD test
tukey_data = []
for strategy in strategies:
    for exp_id, row in metrics_df[metrics_df["strategy"] == strategy].iterrows():
        tukey_data.append({"strategy": strategy, "accuracy": row["accuracy"]})

tukey_df = pd.DataFrame(tukey_data)
tukey_result = pairwise_tukeyhsd(tukey_df["accuracy"], tukey_df["strategy"], alpha=0.05)

print("\n📊 Tukey HSD Post-hoc Test:")
print(tukey_result)

In [ ]:
# Visualize strategy progression
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = ["accuracy", "macro_f1", "mcc", "error_rate"]
titles = ["Accuracy", "Macro-F1", "Matthews Correlation Coefficient", "Error Rate"]
colors = ["#3498db", "#2ecc71", "#9b59b6", "#e74c3c"]

for idx, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]

    strategy_means = []
    strategy_stds = []

    for strategy in strategies:
        values = metrics_df[metrics_df["strategy"] == strategy][metric].values
        strategy_means.append(np.mean(values))
        strategy_stds.append(np.std(values))

    x_pos = np.arange(len(strategies))
    bars = ax.bar(
        x_pos,
        strategy_means,
        yerr=strategy_stds,
        capsize=10,
        color=colors[idx],
        alpha=0.7,
        edgecolor="black",
        linewidth=1.5,
    )

    # Add value labels
    for i, (bar, mean, std) in enumerate(zip(bars, strategy_means, strategy_stds)):
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + std + 0.01,
            f"{mean:.4f}",
            ha="center",
            va="bottom",
            fontsize=10,
            weight="bold",
        )

    ax.set_xlabel("Prompting Strategy", fontsize=12, weight="bold")
    ax.set_ylabel(title, fontsize=12, weight="bold")
    ax.set_title(f"{title} Progression Across Strategies", fontsize=14, weight="bold")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(strategies, rotation=45, ha="right")
    ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig("strategy_effectiveness_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n✓ Strategy effectiveness visualization saved")

## 4. Research Question 2: Model Comparison

**Which model performs best across different prompting strategies?**

In [ ]:
print("\n" + "=" * 100)
print("🤖 MODEL COMPARISON ANALYSIS")
print("=" * 100)

# Group by model
model_analysis = (
    metrics_df.groupby("model")
    .agg(
        {
            "accuracy": ["mean", "std", "min", "max"],
            "macro_f1": ["mean", "std", "min", "max"],
            "mcc": ["mean", "std", "min", "max"],
            "error_rate": ["mean", "std", "min", "max"],
        }
    )
    .round(4)
)

print("\nPerformance by Model:")
display(model_analysis)

# Two-way ANOVA: Strategy × Model interaction
anova_data = metrics_df[["strategy", "model", "accuracy"]].copy()
model = ols(
    "accuracy ~ C(strategy) + C(model) + C(strategy):C(model)", data=anova_data
).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("\n📈 Two-Way ANOVA (Strategy × Model):")
display(anova_table)

# Interpretation
print("\n💡 Interpretation:")
if anova_table.loc["C(strategy)", "PR(>F)"] < 0.05:
    print("  ✓ Strategy has a statistically significant effect on accuracy")
else:
    print("  ✗ Strategy does not have a significant effect on accuracy")

if anova_table.loc["C(model)", "PR(>F)"] < 0.05:
    print("  ✓ Model has a statistically significant effect on accuracy")
else:
    print("  ✗ Model does not have a significant effect on accuracy")

if anova_table.loc["C(strategy):C(model)", "PR(>F)"] < 0.05:
    print("  ✓ Significant interaction between strategy and model")
else:
    print("  ✗ No significant interaction between strategy and model")

In [ ]:
# Model comparison heatmap
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

models = ["Gemini 2.0 Flash", "Gemini 1.5 Flash", "Llama-3.3-70B"]
metrics_to_compare = ["accuracy", "macro_f1", "mcc"]
metric_titles = ["Accuracy", "Macro-F1", "MCC"]

for idx, (metric, title) in enumerate(zip(metrics_to_compare, metric_titles)):
    ax = axes[idx]

    # Create pivot table
    pivot_data = metrics_df.pivot_table(
        values=metric, index="strategy", columns="model", aggfunc="mean"
    )
    pivot_data = pivot_data.reindex(strategies)[models]

    # Create heatmap
    sns.heatmap(
        pivot_data,
        annot=True,
        fmt=".4f",
        cmap="RdYlGn",
        ax=ax,
        cbar_kws={"label": title},
        vmin=pivot_data.min().min(),
        vmax=pivot_data.max().max(),
    )
    ax.set_title(f"{title}: Strategy × Model", fontsize=14, weight="bold")
    ax.set_xlabel("Model", fontsize=12)
    ax.set_ylabel("Strategy", fontsize=12)

plt.tight_layout()
plt.savefig("model_comparison_heatmap.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n✓ Model comparison heatmap saved")

## 5. Research Question 3: Cost-Benefit Analysis

**What are the trade-offs between performance gains and implementation complexity?**

In [ ]:
print("\n" + "=" * 100)
print("💰 COST-BENEFIT ANALYSIS")
print("=" * 100)

# Define complexity scores (1-10 scale)
complexity_scores = {
    "Zero-Shot": 1,
    "Few-Shot": 3,
    "Chain-of-Thought": 6,
    "Tree-of-Thought": 9,
}

# Calculate efficiency: Performance / Complexity
metrics_df["complexity"] = metrics_df["strategy"].map(complexity_scores)
metrics_df["efficiency_accuracy"] = metrics_df["accuracy"] / metrics_df["complexity"]
metrics_df["efficiency_f1"] = metrics_df["macro_f1"] / metrics_df["complexity"]

print("\nEfficiency Rankings (Performance per Complexity Unit):")
print("\nBy Accuracy Efficiency:")
display(
    metrics_df[
        ["experiment", "strategy", "accuracy", "complexity", "efficiency_accuracy"]
    ]
    .sort_values("efficiency_accuracy", ascending=False)
    .round(4)
)

print("\nBy F1 Efficiency:")
display(
    metrics_df[["experiment", "strategy", "macro_f1", "complexity", "efficiency_f1"]]
    .sort_values("efficiency_f1", ascending=False)
    .round(4)
)

# Performance gain analysis
baseline_accuracy = metrics_df[metrics_df["strategy"] == "Zero-Shot"]["accuracy"].mean()
baseline_f1 = metrics_df[metrics_df["strategy"] == "Zero-Shot"]["macro_f1"].mean()

strategy_gains = []
for strategy in strategies:
    strategy_data = metrics_df[metrics_df["strategy"] == strategy]
    accuracy_gain = (
        (strategy_data["accuracy"].mean() - baseline_accuracy) / baseline_accuracy
    ) * 100
    f1_gain = ((strategy_data["macro_f1"].mean() - baseline_f1) / baseline_f1) * 100
    complexity = complexity_scores[strategy]

    strategy_gains.append(
        {
            "Strategy": strategy,
            "Complexity": complexity,
            "Accuracy_Gain_%": accuracy_gain,
            "F1_Gain_%": f1_gain,
            "Efficiency_Score": (accuracy_gain + f1_gain) / (2 * complexity),
        }
    )

gains_df = pd.DataFrame(strategy_gains)

print("\n📊 Performance Gains vs. Baseline (Zero-Shot):")
display(gains_df.round(4))

In [ ]:
# Cost-benefit visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Plot 1: Performance vs. Complexity
ax1 = axes[0]
for model in models:
    model_data = metrics_df[metrics_df["model"] == model]
    ax1.scatter(
        model_data["complexity"],
        model_data["accuracy"],
        s=200,
        alpha=0.7,
        label=model,
        edgecolors="black",
        linewidths=2,
    )

ax1.set_xlabel("Implementation Complexity (1-10)", fontsize=12, weight="bold")
ax1.set_ylabel("Accuracy", fontsize=12, weight="bold")
ax1.set_title("Performance vs. Complexity Trade-off", fontsize=14, weight="bold")
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Efficiency comparison
ax2 = axes[1]
strategy_efficiency = (
    metrics_df.groupby("strategy")["efficiency_accuracy"].mean().reindex(strategies)
)
bars = ax2.bar(
    range(len(strategies)),
    strategy_efficiency,
    color=["#3498db", "#2ecc71", "#f39c12", "#e74c3c"],
    alpha=0.7,
    edgecolor="black",
    linewidth=1.5,
)

for i, (bar, val) in enumerate(zip(bars, strategy_efficiency)):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.005,
        f"{val:.4f}",
        ha="center",
        va="bottom",
        fontsize=10,
        weight="bold",
    )

ax2.set_xlabel("Prompting Strategy", fontsize=12, weight="bold")
ax2.set_ylabel("Efficiency Score (Accuracy / Complexity)", fontsize=12, weight="bold")
ax2.set_title("Strategy Efficiency Comparison", fontsize=14, weight="bold")
ax2.set_xticks(range(len(strategies)))
ax2.set_xticklabels(strategies, rotation=45, ha="right")
ax2.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig("cost_benefit_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n✓ Cost-benefit analysis visualization saved")

## 6. Research Question 4: Statistical Significance

**Are performance differences statistically meaningful or due to chance?**

In [ ]:
print("\n" + "=" * 100)
print("📊 STATISTICAL SIGNIFICANCE TESTING")
print("=" * 100)

# Pairwise comparisons between strategies
print("\n1. Pairwise t-tests between consecutive strategies:")
print("=" * 80)

comparisons = [
    ("Zero-Shot", "Few-Shot"),
    ("Few-Shot", "Chain-of-Thought"),
    ("Chain-of-Thought", "Tree-of-Thought"),
]

for strategy1, strategy2 in comparisons:
    acc1 = metrics_df[metrics_df["strategy"] == strategy1]["accuracy"].values
    acc2 = metrics_df[metrics_df["strategy"] == strategy2]["accuracy"].values

    t_stat, p_val = ttest_ind(acc1, acc2)

    print(f"\n{strategy1} vs. {strategy2}:")
    print(f"  Mean Accuracy: {acc1.mean():.4f} vs. {acc2.mean():.4f}")
    print(f"  Difference: {(acc2.mean() - acc1.mean()):.4f}")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.4f}")
    print(f"  Significant: {'Yes ✓' if p_val < 0.05 else 'No ✗'} (α=0.05)")

# Model comparisons
print("\n\n2. Pairwise t-tests between models:")
print("=" * 80)

model_pairs = [
    ("Gemini 2.0 Flash", "Gemini 1.5 Flash"),
    ("Gemini 1.5 Flash", "Llama-3.3-70B"),
    ("Gemini 2.0 Flash", "Llama-3.3-70B"),
]

for model1, model2 in model_pairs:
    acc1 = metrics_df[metrics_df["model"] == model1]["accuracy"].values
    acc2 = metrics_df[metrics_df["model"] == model2]["accuracy"].values

    t_stat, p_val = ttest_ind(acc1, acc2)

    print(f"\n{model1} vs. {model2}:")
    print(f"  Mean Accuracy: {acc1.mean():.4f} vs. {acc2.mean():.4f}")
    print(f"  Difference: {(acc2.mean() - acc1.mean()):.4f}")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.4f}")
    print(f"  Significant: {'Yes ✓' if p_val < 0.05 else 'No ✗'} (α=0.05)")


# Effect size (Cohen's d)
def cohens_d(group1, group2):
    """Calculate Cohen's d effect size"""
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    return (np.mean(group1) - np.mean(group2)) / pooled_std


print("\n\n3. Effect sizes (Cohen's d):")
print("=" * 80)
print(
    "Interpretation: |d| < 0.2 (small), 0.2-0.5 (medium), 0.5-0.8 (large), > 0.8 (very large)"
)

for strategy1, strategy2 in comparisons:
    acc1 = metrics_df[metrics_df["strategy"] == strategy1]["accuracy"].values
    acc2 = metrics_df[metrics_df["strategy"] == strategy2]["accuracy"].values
    d = cohens_d(acc2, acc1)

    magnitude = (
        "small"
        if abs(d) < 0.2
        else "medium"
        if abs(d) < 0.5
        else "large"
        if abs(d) < 0.8
        else "very large"
    )
    print(f"\n{strategy1} → {strategy2}: d = {d:.4f} ({magnitude})")

## 7. Research Question 5: Production Deployment Recommendations

**Which approach should be deployed in production systems?**

In [ ]:
print("\n" + "=" * 100)
print("🎯 PRODUCTION DEPLOYMENT RECOMMENDATIONS")
print("=" * 100)


# Multi-criteria scoring
def calculate_production_score(row):
    """
    Calculate production readiness score based on:
    - Performance (40%): accuracy + macro_f1 + mcc
    - Reliability (30%): 1 - error_rate
    - Efficiency (30%): performance / complexity
    """
    performance = (row["accuracy"] + row["macro_f1"] + row["mcc"]) / 3
    reliability = 1 - row["error_rate"]
    efficiency = row["efficiency_accuracy"]

    return (performance * 0.4) + (reliability * 0.3) + (efficiency * 0.3)


metrics_df["production_score"] = metrics_df.apply(calculate_production_score, axis=1)

print("\n📊 Production Readiness Rankings:")
rankings = metrics_df[
    [
        "experiment",
        "model",
        "strategy",
        "accuracy",
        "macro_f1",
        "mcc",
        "error_rate",
        "efficiency_accuracy",
        "production_score",
    ]
].sort_values("production_score", ascending=False)
display(rankings.round(4))

# Top recommendations
print("\n\n🏆 TOP 3 RECOMMENDATIONS FOR PRODUCTION:")
print("=" * 80)

for i, (idx, row) in enumerate(rankings.head(3).iterrows(), 1):
    print(f"\n{i}. {row['experiment']}: {row['model']} with {row['strategy']}")
    print(f"   • Accuracy: {row['accuracy']:.4f}")
    print(f"   • Macro-F1: {row['macro_f1']:.4f}")
    print(f"   • MCC: {row['mcc']:.4f}")
    print(f"   • Error Rate: {row['error_rate']:.4f}")
    print(f"   • Efficiency: {row['efficiency_accuracy']:.4f}")
    print(f"   • Production Score: {row['production_score']:.4f}")

# Decision matrix
print("\n\n💡 DEPLOYMENT DECISION MATRIX:")
print("=" * 80)

print("\n🚀 High Performance Required:")
best_perf = metrics_df.nlargest(1, "accuracy").iloc[0]
print(
    f"   Recommended: {best_perf['experiment']} ({best_perf['model']}, {best_perf['strategy']})"
)
print(
    f"   Accuracy: {best_perf['accuracy']:.4f} | F1: {best_perf['macro_f1']:.4f} | MCC: {best_perf['mcc']:.4f}"
)

print("\n⚡ Cost-Efficiency Priority:")
best_eff = metrics_df.nlargest(1, "efficiency_accuracy").iloc[0]
print(
    f"   Recommended: {best_eff['experiment']} ({best_eff['model']}, {best_eff['strategy']})"
)
print(
    f"   Efficiency: {best_eff['efficiency_accuracy']:.4f} | Accuracy: {best_eff['accuracy']:.4f}"
)

print("\n🎯 Balanced Approach:")
best_balanced = metrics_df.nlargest(1, "production_score").iloc[0]
print(
    f"   Recommended: {best_balanced['experiment']} ({best_balanced['model']}, {best_balanced['strategy']})"
)
print(f"   Production Score: {best_balanced['production_score']:.4f}")

print("\n🔒 Reliability Priority (Lowest Error Rate):")
best_reliable = metrics_df.nsmallest(1, "error_rate").iloc[0]
print(
    f"   Recommended: {best_reliable['experiment']} ({best_reliable['model']}, {best_reliable['strategy']})"
)
print(
    f"   Error Rate: {best_reliable['error_rate']:.4f} | Accuracy: {best_reliable['accuracy']:.4f}"
)

In [ ]:
# Production recommendation visualization
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Plot 1: Production scores
ax1 = axes[0, 0]
sorted_df = metrics_df.sort_values("production_score", ascending=False)
colors_map = {
    "Zero-Shot": "#3498db",
    "Few-Shot": "#2ecc71",
    "Chain-of-Thought": "#f39c12",
    "Tree-of-Thought": "#e74c3c",
}
colors_list = [colors_map[s] for s in sorted_df["strategy"]]

bars = ax1.barh(
    range(len(sorted_df)),
    sorted_df["production_score"],
    color=colors_list,
    alpha=0.7,
    edgecolor="black",
    linewidth=1.5,
)
ax1.set_yticks(range(len(sorted_df)))
ax1.set_yticklabels(sorted_df["experiment"])
ax1.set_xlabel("Production Readiness Score", fontsize=12, weight="bold")
ax1.set_title("Production Readiness Rankings", fontsize=14, weight="bold")
ax1.grid(axis="x", alpha=0.3)

# Plot 2: Performance vs. Efficiency scatter
ax2 = axes[0, 1]
for strategy in strategies:
    strategy_data = metrics_df[metrics_df["strategy"] == strategy]
    ax2.scatter(
        strategy_data["efficiency_accuracy"],
        strategy_data["accuracy"],
        s=200,
        alpha=0.7,
        label=strategy,
        color=colors_map[strategy],
        edgecolors="black",
        linewidths=2,
    )

ax2.set_xlabel("Efficiency (Accuracy/Complexity)", fontsize=12, weight="bold")
ax2.set_ylabel("Accuracy", fontsize=12, weight="bold")
ax2.set_title("Performance vs. Efficiency", fontsize=14, weight="bold")
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: Multi-criteria radar chart for top 3
ax3 = axes[1, 0]
ax3.axis("off")

top3 = rankings.head(3)
categories = ["Accuracy", "Macro-F1", "MCC", "Reliability", "Efficiency"]
num_vars = len(categories)

angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

ax_radar = plt.subplot(2, 2, 3, projection="polar")

for idx, (_, row) in enumerate(top3.iterrows()):
    values = [
        row["accuracy"],
        row["macro_f1"],
        row["mcc"],
        1 - row["error_rate"],
        row["efficiency_accuracy"],
    ]
    values += values[:1]

    ax_radar.plot(
        angles,
        values,
        "o-",
        linewidth=2,
        label=f"{row['experiment']}: {row['strategy']}",
        markersize=8,
    )
    ax_radar.fill(angles, values, alpha=0.15)

ax_radar.set_xticks(angles[:-1])
ax_radar.set_xticklabels(categories, fontsize=10)
ax_radar.set_ylim(0, 1)
ax_radar.set_title(
    "Top 3 Configurations: Multi-Criteria Comparison",
    fontsize=14,
    weight="bold",
    pad=20,
)
ax_radar.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1), fontsize=9)
ax_radar.grid(True)

# Plot 4: Strategy-wise box plots
ax4 = axes[1, 1]
strategy_data = [
    metrics_df[metrics_df["strategy"] == s]["accuracy"].values for s in strategies
]
bp = ax4.boxplot(
    strategy_data,
    labels=strategies,
    patch_artist=True,
    boxprops=dict(facecolor="#3498db", alpha=0.7),
    medianprops=dict(color="red", linewidth=2),
    whiskerprops=dict(linewidth=1.5),
    capprops=dict(linewidth=1.5),
)

ax4.set_xlabel("Prompting Strategy", fontsize=12, weight="bold")
ax4.set_ylabel("Accuracy", fontsize=12, weight="bold")
ax4.set_title("Accuracy Distribution by Strategy", fontsize=14, weight="bold")
ax4.set_xticklabels(strategies, rotation=45, ha="right")
ax4.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig("production_recommendations.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n✓ Production recommendations visualization saved")

## 8. Key Findings Summary

In [ ]:
print("\n" + "=" * 100)
print("📋 KEY FINDINGS SUMMARY")
print("=" * 100)

# Calculate key statistics
best_overall = metrics_df.nlargest(1, "accuracy").iloc[0]
most_efficient = metrics_df.nlargest(1, "efficiency_accuracy").iloc[0]
best_production = metrics_df.nlargest(1, "production_score").iloc[0]

strategy_improvement = (
    (
        metrics_df[metrics_df["strategy"] == "Tree-of-Thought"]["accuracy"].mean()
        - metrics_df[metrics_df["strategy"] == "Zero-Shot"]["accuracy"].mean()
    )
    / metrics_df[metrics_df["strategy"] == "Zero-Shot"]["accuracy"].mean()
) * 100

print(f"""
1. BEST OVERALL PERFORMER:
   • Configuration: {best_overall["experiment"]} ({best_overall["model"]}, {best_overall["strategy"]})
   • Accuracy: {best_overall["accuracy"]:.4f}
   • Macro-F1: {best_overall["macro_f1"]:.4f}
   • MCC: {best_overall["mcc"]:.4f}

2. MOST EFFICIENT:
   • Configuration: {most_efficient["experiment"]} ({most_efficient["model"]}, {most_efficient["strategy"]})
   • Efficiency Score: {most_efficient["efficiency_accuracy"]:.4f}
   • Accuracy: {most_efficient["accuracy"]:.4f}

3. PRODUCTION RECOMMENDED:
   • Configuration: {best_production["experiment"]} ({best_production["model"]}, {best_production["strategy"]})
   • Production Score: {best_production["production_score"]:.4f}
   • Balanced across performance, reliability, and efficiency

4. STRATEGY PROGRESSION:
   • Zero-Shot → Tree-of-Thought improvement: {strategy_improvement:.2f}%
   • Statistical significance: {"Confirmed" if p_value < 0.05 else "Not confirmed"}

5. MODEL PERFORMANCE:
   • Best model overall: {metrics_df.groupby("model")["accuracy"].mean().idxmax()}
   • Average accuracy: {metrics_df.groupby("model")["accuracy"].mean().max():.4f}

6. IMPLEMENTATION RECOMMENDATIONS:
   • For production deployment: {best_production["strategy"]} with {best_production["model"]}
   • For research/experimentation: {best_overall["strategy"]} with {best_overall["model"]}
   • For cost-sensitive applications: {most_efficient["strategy"]} with {most_efficient["model"]}
""")

print("=" * 100)
print("✓ COMPREHENSIVE COMPARATIVE ANALYSIS COMPLETE")
print("=" * 100)

## 9. Export Results

In [ ]:
# Save comprehensive analysis results
output_file = "comprehensive_comparative_analysis_results.csv"
metrics_df.to_csv(output_file, index=False)

print(f"\n✓ Comprehensive analysis results saved to: {output_file}")
print(f"  Total experiments analyzed: {len(metrics_df)}")
print(f"  Metrics computed: {len(metrics_df.columns)}")
print(f"  Visualizations generated: 4 figures")